# Symulowane wyżarzanie
## Laboratorium 4 - Metody Obliczeniowe w Nauce i Technice

In [2]:
import numpy as np
import copy

In [32]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __str__(self):
        return f"Point({self.x}, {self.y})"
    
    @staticmethod
    def print_1d(array): 
        print("[", end='')
        for p in array: 
            print(p, end=", ")
        print("\b\b]")
    
    @staticmethod
    def print_2d(array): 
        print("[")
        for row in array:
            Point.print_1d(row)
        print("]")
        

def generate_points_arrays(n):
    P = []
    normal_parameters = [(4, 2), (8, 8), (0, 6), (512, 64)]
    
    # List of point tuples: first is bottom left, second is upper right corner of a region
    group_point_regions = [
        (Point(0, 0), Point(1, 1)),
        (Point(2, 0), Point(3, 1)),
        (Point(4, 0), Point(5, 1)),
        (Point(0, 2), Point(1, 3)),
        (Point(2, 2), Point(3, 3)),
        (Point(4, 2), Point(5, 3)),
        (Point(0, 5), Point(1, 6)),
        (Point(2, 4), Point(3, 5)),
        (Point(5, 5), Point(6, 6))
    ]
    m = len(group_point_regions)
    
    # Uniform distribution
    P.append([Point(np.random.randint(0, 2 * n**0.5), np.random.randint(0, 2 * n**0.5)) for _ in range(n)])
    
    # Normal distribution (4 different set of parameters)
    for l, s in normal_parameters:
        x = np.random.normal(l, s, n)
        y = np.random.normal(l, s, n)
        P.append([Point(x[i], y[i]) for i in range(n)])
        
    # 9 group of points
    x = np.random.uniform(group_point_regions[0][0].x, group_point_regions[0][1].x, n // m + n % m)
    y = np.random.uniform(group_point_regions[0][0].y, group_point_regions[0][1].y, n // m + n % m)
    for i in range(1, m):
        x = np.concatenate((x, np.random.uniform(group_point_regions[i][0].x, group_point_regions[i][1].x, n // m)))
        y = np.concatenate((y, np.random.uniform(group_point_regions[i][0].y, group_point_regions[i][1].y, n // m)))
    P.append([Point(x[i], y[i]) for i in range(n)])
    
    return np.array(P)


P = generate_points_arrays(100)
Point.print_2d(P)

[
[Point(3, 16), Point(9, 8), Point(19, 0), Point(17, 19), Point(5, 18), Point(12, 16), Point(5, 1), Point(5, 17), Point(5, 19), Point(9, 0), Point(14, 19), Point(1, 4), Point(16, 17), Point(15, 10), Point(3, 13), Point(15, 7), Point(8, 8), Point(19, 5), Point(12, 14), Point(6, 1), Point(12, 16), Point(19, 14), Point(0, 16), Point(0, 2), Point(1, 8), Point(7, 5), Point(15, 0), Point(0, 1), Point(13, 15), Point(9, 9), Point(9, 14), Point(5, 7), Point(17, 10), Point(7, 7), Point(19, 14), Point(8, 15), Point(2, 3), Point(3, 0), Point(15, 16), Point(8, 12), Point(17, 9), Point(2, 2), Point(10, 10), Point(19, 8), Point(3, 10), Point(16, 4), Point(15, 15), Point(16, 2), Point(16, 14), Point(11, 17), Point(5, 12), Point(9, 10), Point(18, 10), Point(13, 8), Point(16, 14), Point(14, 2), Point(11, 6), Point(12, 18), Point(4, 16), Point(1, 5), Point(4, 18), Point(11, 3), Point(12, 18), Point(15, 8), Point(9, 8), Point(4, 5), Point(19, 4), Point(9, 10), Point(18, 7), Point(15, 9), Point(8, 11), Po

In [34]:
class SimulatedAnnealing:
    def __init__(self, initial_solution, gamma, initial_T, max_iterations):
        self.initial_solution = initial_solution
        self.gamma = gamma
        self.T = initial_T
        self.max_iterations = max_iterations
        
    def run(self):
        state, state_cost = copy.deepcopy(self.initial_solution), self.get_cost(self.initial_solution)
        solution, solution_cost = copy.deepcopy(self.initial_solution), self.get_cost(self.initial_solution)
        
        for i in range(self.max_iterations):
            if self.T <= 10**-3:
                break

            new_state = self.neighbour(state)
            new_state_cost = self.get_cost(new_state)

            if self.P(new_state_cost, state_cost) <= np.random.uniform(0, 1):
                state, state_cost = new_state, self.get_cost(new_state)
                if self.get_cost(solution) < self.get_cost(state):
                   solution, solution_cost = copy.deepcopy(state), state_cost
                
        return solution, solution_cost
    
    def distance(self, p1, p2):
        return ((p2.x - p1.x)**2 + (p2.y - p1.y)**2)**.5 
    
    def get_cost(self, state):
        acc = 0.0
        for i, j in [(i, (i + 1) % len(state)) for i in range(len(state))]:
            acc += self.distance(state[i], state[j])
            
        return acc
        
    def neighbour(self, state):
        state = state.copy()
        i, j = np.random.default_rng().choice(len(state), 2)
        state[i], state[j] = state[j], state[i]
        
        return state
        
    def P(self, new_state_cost, state_cost):
        if new_state_cost < state_cost:
            return 1
        return np.exp((state_cost - new_state_cost) / self.T)
        

for p in P:
    res, total_distance = SimulatedAnnealing(p, .95, 2**10, 20000).run()
    # Point.print_1d(res)
    print(total_distance)
    
print()
for p in P:
    res, total_distance = SimulatedAnnealing(p, .95, 2**10, 20000).run()
    # Point.print_1d(res)
    print(total_distance)

1272.6329933304428
434.0110791148078
1734.2291853749812
1346.5317642950063
14842.467047728962
385.04998685561617

1303.5444874581635
456.0886238447416
1726.1081656855565
1355.989855585912
14866.79645998747
385.2975554059754
